In [1]:
using JuMP, Gurobi, DataFrames, CSV, Distributions, Plots

In [ ]:
#load summer_months
#summer_months = CSV.read("summer_months_new.csv", DataFrame);

In [ ]:
#pick 100 sample days from summer_months
#sample_days = sample(1:244, 100, replace=false);

In [ ]:
# filter the columns in summer_months from sample_days
#sample_days = summer_months[sample_days, :];

In [ ]:
#select!(sample_days, Not(:Date));

In [ ]:
#select!(sample_days, Not(:Index))

In [ ]:
#save sample_days to csv
#CSV.write("sample_days_new.csv", sample_days);

In [2]:
sample_days = CSV.read("sample_days_new.csv", DataFrame);

In [12]:
realtime = CSV.read("19_Jun_2019_realtime.csv", DataFrame);

In [4]:
# read dayahead prices
dayahead = CSV.read("19_Jun_2019_dayahead.csv", DataFrame); # for 19th June 2019
#dayahead = CSV.read("30_July_2019_dayahead.csv", DataFrame); # for 30th July 2019

Row,Hours,LZ_HOUSTON
,Int64,Float64
1,0,16.68
2,1,15.21
3,2,15.22
4,3,14.44
5,4,14.46
6,5,14.05
7,6,16.21
8,7,16.93
9,8,19.37


In [5]:
dayaheadprices = dayahead.LZ_HOUSTON;

In [6]:
samplematrix = Matrix(sample_days);

In [7]:
# add dayaheadprices to samplematrix for each row
for i in 1:size(samplematrix, 1)
    samplematrix[i, :] = samplematrix[i,:] .+ dayaheadprices
end

In [8]:
for i in 1:size(samplematrix, 1)
    for j in 1:size(samplematrix, 2)
        if samplematrix[i, j] < -251
            samplematrix[i, j] = samplematrix[i, j] * -1
        end
    end
end

In [9]:
size(samplematrix)

(100, 24)

In [10]:
maximum(samplematrix)

1599.8999999999999

In [11]:
samplematrix = round.(samplematrix, digits=2);

In [13]:
using JuMP, Gurobi, DataFrames

function optimize_battery_operations_cvar_ver3(prices::Matrix, battery_capacity::Float64, max_power::Float64, charging_efficiency::Float64, discharging_efficiency::Float64, α::Float64, β::Float64)
    # Number of time periods (assuming 24-hour price scenarios)
    T = size(prices, 2)
    num_scenarios = size(prices, 1)

    # Create DataFrames to store the results
    results = DataFrame(scenario = Int[], hour = Int[], price = Float64[], soc = Float64[], charge = Float64[], discharge = Float64[])
    objective_values = DataFrame(scenario = Int[], profit = Float64[])

    # Initialize the model with the Gurobi solver
    model = Model(Gurobi.Optimizer)

    # Decision variables
    @variable(model, 0 <= soc[1:T] <= battery_capacity)  # State of charge
    @variable(model, 0 <= charge[1:T] <= max_power)  # Charge
    @variable(model, 0 <= discharge[1:T] <= max_power)  # Discharge
    @variable(model, ζ >=0)
    @variable(model, z[1:num_scenarios] >=0)
    @variable(model, u[1:T], Bin)

    # Objective function: Maximize profit
    @objective(model, Max, sum(0.01*sum(prices[s, t] * (discharge[t] - charge[t]) for t in 1:T, s in 1:num_scenarios)) - β * (ζ + (1/(1-α)) * sum(0.01 * z[s] for s in 1:num_scenarios)))

    # Constraints
    @constraint(model, [s in 1:num_scenarios], soc[1] == 0)  # Initial SOC
    @constraint(model, [s in 1:num_scenarios], discharge[1] == 0)  # Prevent discharging at the first time step
    for t in 2:T
        for s in 1:num_scenarios
            @constraint(model, soc[t] == soc[t - 1] + charging_efficiency * charge[t - 1] - (1 / discharging_efficiency) * discharge[t - 1])  # SOC dynamics
            @constraint(model, soc[t]>=(discharge[t] - charge[t]))
            @constraint(model, z[s] >= sum(prices[s,t]*(discharge[t]-charge[t]))-ζ)
            @constraint(model, discharge[t]-max_power*u[t]<=0)
            @constraint(model, charge[t]-max_power*(1-u[t])<=0)
        end
    end

    # Solve the optimization problem
    optimize!(model)
    
    #save ζ value
    ζ_value = value(ζ)

    # Save the results
    for s in 1:num_scenarios
        for t in 1:T
            push!(results, (s, t, prices[s, t], value(soc[t]), value(charge[t]), value(discharge[t])))
        end
        push!(objective_values, (s, objective_value(model)))
    end
    return results, objective_values, ζ_value
end
battery_capacity = 100.0
max_power = 30.0
charging_efficiency = 0.95
discharging_efficiency = 0.95
α = 0.95
β = 0.3

0.3

In [14]:
results_df, objective_values_df, zetaval = optimize_battery_operations_cvar_ver3(samplematrix, battery_capacity, max_power, charging_efficiency, discharging_efficiency, α, β)

Set parameter Username
Academic license - for non-commercial use only - expires 2024-04-16
Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (win64)

CPU model: 12th Gen Intel(R) Core(TM) i7-1265U, instruction set [SSE2|AVX|AVX2]
Thread count: 10 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 11700 rows, 197 columns and 34700 nonzeros
Model fingerprint: 0x4628389d
Variable types: 173 continuous, 24 integer (24 binary)
Coefficient statistics:
  Matrix range     [2e-01, 2e+03]
  Objective range  [6e-02, 1e+02]
  Bounds range     [3e+01, 1e+02]
  RHS range        [3e+01, 3e+01]
Found heuristic solution: objective -0.0000000
Presolve removed 9333 rows and 6 columns
Presolve time: 0.02s
Presolved: 2367 rows, 191 columns, 9278 nonzeros
Variable types: 169 continuous, 22 integer (22 binary)

Root relaxation: objective 2.104161e+03, 44 iterations, 0.01 seconds (0.01 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl 

(2400×6 DataFrame
  Row │ scenario  hour   price    soc       charge   discharge 
      │ Int64     Int64  Float64  Float64   Float64  Float64   
──────┼────────────────────────────────────────────────────────
    1 │        1      1    16.18    0.0      0.0       0.0
    2 │        1      2    15.45    0.0      0.0       0.0
    3 │        1      3    16.21    0.0      0.0       0.0
    4 │        1      4    15.64    0.0     15.2632    0.0
    5 │        1      5    15.01   14.5     30.0       0.0
    6 │        1      6    13.9    43.0     30.0       0.0
    7 │        1      7    15.98   71.5     30.0       0.0
    8 │        1      8    17.35  100.0      0.0       0.0
  ⋮   │    ⋮        ⋮       ⋮        ⋮         ⋮         ⋮
 2394 │      100     18    74.94   45.4819   0.0       6.10198
 2395 │      100     19    66.53   39.0587   0.0       5.54142
 2396 │      100     20    51.24   33.2257   0.0      13.2566
 2397 │      100     21   107.8    19.2714   0.0      18.3078
 2398 │  

In [15]:
zetaval

5957.9084752475865

In [16]:
profit = sum(dayahead.LZ_HOUSTON.*(results_df[results_df.scenario .== 1, :].discharge.-results_df[results_df.scenario .== 1, :].charge))

3944.855450063965

In [21]:
#save results_df to csv
CSV.write("Results/results_df_cvar_0.3_reformulated_ver3.csv", results_df)

"Results/results_df_cvar_0.3_reformulated_ver3.csv"